In [1]:
import pandas as pd
import numpy as np
import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader


In [5]:
torch.argmax(torch.tensor([[0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3848, 0.1572],
        [0.2285, 0.2285, 0.3867, 0.1562]], dtype=torch.bfloat16),dim=-1).shape

torch.Size([10])

In [2]:
df = pd.read_csv("../data/twitter_training.csv",names=['a','company','label','text'])

In [3]:
df.head()


,a,company,label,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
df.label.unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [5]:
def mapper(col):
    unique=df.col.unique()
    

In [6]:
uni= df.label.unique()
d={uni[i]: i for i in range(len(uni))}

In [7]:
*uni

SyntaxError: can't use starred expression here (2278755055.py, line 1)

In [8]:
df["labeli"]=df["label"].map(d)

In [9]:
df.sample(10)

,a,company,label,text,labeli
32616,7595,LeagueOfLegends,Negative,@discord @RiotSupport @Mortdog @kami_banani...,2
21126,4017,CS-GO,Neutral,Since when did I think renaming the bomb to “T...,1
25573,4790,Google,Negative,.. please update your Google Maps.. It has giv...,2
921,2563,Borderlands,Neutral,The RhandlerR is my first piece of fan made Bo...,1
46056,11905,Verizon,Positive,"Need for speed? Verizon, Ericsson and Qualcomm...",0
25744,819,AssassinsCreed,Negative,@Ubisoft @assassinscreed Please Please Make Yo...,2
22537,4258,CS-GO,Irrelevant,CSGO Except at odds with a pile of shit runnin...,3
44301,11610,Verizon,Negative,Funny how this feature is not HUGE during each...,2
20248,12667,WorldOfCraft,Neutral,I probably just earned the [... Brutally Dedic...,1
25101,4709,Google,Neutral,Our favourite smart home devices that aren't d...,1


In [10]:
tokenizer= tiktoken.get_encoding("gpt2")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   a        74682 non-null  int64 
 1   company  74682 non-null  object
 2   label    74682 non-null  object
 3   text     73996 non-null  object
 4   labeli   74682 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.8+ MB


In [12]:
tokenizer.n_vocab

50257

In [13]:
df[~df.text.isna()].text.isna().sum()

np.int64(0)

In [14]:
class ClassifyData(Dataset):
    def __init__(self,csv,tokenizer,max_length=None, pad_token_id=50256):
        df = self._df_process(csv)
        self.input_ids=[tokenizer.encode(text) for text in df["text"]]
        self.output_ids=[ids for ids in df['labeli']]
        if max_length is None:
           max_length= max([len(i) for i in self.input_ids])
        print(max_length)
        self.input_ids=[
            inputs[:max_length]
            for inputs in self.input_ids
        ]
        self.encoded= [
            inputs+[pad_token_id]*(max_length-len(inputs))
            for inputs in self.input_ids
        ]


    def __len__(self):
        return df.shape[0]

    def __getitem__(self, index):
        return (
           torch.tensor(self.encoded[index],dtype=torch.long),
            torch.tensor(self.output_ids[index],dtype=torch.long)
        )
        
    def _df_process(self,csv) -> pd.DataFrame:
        df = pd.read_csv(csv,names=['a','company','label','text'])
        df=df[~df.text.isna()]
        df=df[~df.label.isna()]
        uni= df.label.unique()
        d={uni[i]: i for i in range(len(uni))}
        self.classes=d
        df["labeli"]=df["label"].map(d)
        return df


        


In [15]:
data= ClassifyData("../data/twitter_training.csv",tokenizer)


346


In [16]:
data[0][0].shape

torch.Size([346])

In [17]:
dl= DataLoader(data,128,True,num_workers=13)

c:\Users\amanm\OneDrive\Documents\code\py\llm_gpt\venvv\Lib\site-packages\torch\utils\data\dataloader.py:626: UserWarning: This DataLoader will create 13 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
from model import Llama2Model
from torch import nn
from config import LLAMA2_CONFIG_7B


ModuleNotFoundError: No module named 'model'

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
device="cuda" if torch.cuda.is_available() else "cpu"
model= Llama2Model(LLAMA2_CONFIG_7B).to(device)
for parms in model.parameters():
    parms.requires_grad=False

model.out_head= nn.Linear(LLAMA2_CONFIG_7B["emb_dim"],len(data.classes))

for parms in model.trf_blocks[-1].parameters():
    parms.requires_grad=True

for parms in model.final_norm.parameters():
    parms.requires_grad=True


In [1]:
di={1:2,2:3}

In [2]:
di[1]

2